In [ ]:
import transformers 
from threading import Thread
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

#Optional: set backend/compile
#You will need to install CUDA kernels apriori
# git clone https://github.com/mobiusml/hqq/
# cd hqq/kernels && python setup_cuda.py install
from hqq.core.quantize import *

In [ ]:
model_path = '/home/cdsw/models/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bitgs8-metaoffload-HQQ'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = HQQModelForCausalLM.from_quantized(model_path)
# HQQLinear.set_backend(HQQBackend.ATEN_BACKPROP)

In [ ]:
def chat_processor(chat, max_new_tokens=100, do_sample=True):
    tokenizer.use_default_system_prompt = False
    streamer = transformers.TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

    generate_params = dict(
        tokenizer("<s> [INST] " + chat + " [/INST] ", return_tensors="pt").to('cuda'),
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=0.90,
        top_k=50,
        temperature= 0.6,
        num_beams=1,
        repetition_penalty=1.2,
    )

    t = Thread(target=model.generate, kwargs=generate_params)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        print(text, end="", flush=True)

    return outputs

In [ ]:
outputs = chat_processor("How do I build a car?", max_new_tokens=1000, do_sample=False)